In [1]:
import pandas as pd
import typing as t
import numpy as np

In [3]:
data = pd.read_csv("../data/GrandEst.csv", index_col=0, sep=';')

/home/cindy/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,4,11,12,13,15,18,21,24,39,40,48,49,54,55,56,70,71,77,80,83,84,89,91,92,93,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/cindy/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data.columns

Index(['NUMMR', 'ACHLR', 'AEMM', 'AEMMR', 'AGED', 'AGER20', 'AGEREV',
       'AGEREVQ', 'ANAI', 'ANARR', 'ANEM', 'ANEMR', 'APAF', 'ARRIVR', 'ASCEN',
       'BAIN', 'BATI', 'CATL', 'CATPC', 'CHAU', 'CHFL', 'CHOS', 'CLIM', 'CMBL',
       'COUPLE', 'CS1', 'CS2', 'CS3', 'CUIS', 'DEPT', 'DEROU', 'DIPL_15',
       'EAU', 'EGOUL', 'ELEC', 'EMPL', 'EPCI', 'ETUD', 'GARL', 'HLML',
       'ILETUD', 'ILETUU', 'ILT', 'ILTUU', 'IMMI', 'INAI', 'INAT', 'INFAM',
       'INPER', 'INPERF', 'IPONDI', 'IRAN', 'IRANUU', 'LIENF', 'LPRF', 'LPRM',
       'METRODOM', 'MOCO', 'MODV', 'NA38', 'NA88', 'NAF08', 'NAIDT', 'NAT13',
       'NAT49', 'NATC', 'NATN12', 'NATN49', 'NATNC', 'NBPI', 'NPERR', 'NUMF',
       'ORIDT', 'PNAI12', 'PROF', 'RECH', 'SANI', 'SANIDOM', 'SEXE', 'SFM',
       'STAT', 'STAT_CONJ', 'STOCD', 'SURF', 'TACT', 'TACTD16', 'TP', 'TRANS',
       'TYPC', 'TYPFC', 'TYPL', 'TYPMD', 'TYPMR', 'UR', 'VOIT', 'WC'],
      dtype='object')

In [4]:
to_study = ["ACHLR", "AGER20", "CMBL", "CS1", "DIPL_15", "HLML", "INAT", "INPER", "MODV", "SURF", "TYPL", "UR"]

In [5]:
our_data = data[to_study]

In [6]:
our_data.reset_index(drop=True, inplace=True)

In [7]:
our_data

,ACHLR,AGER20,CMBL,CS1,DIPL_15,HLML,INAT,INPER,MODV,SURF,TYPL,UR
0,3,64,2,8,A,1,11,2,40,7,2,1
1,3,79,2,7,A,1,11,2,40,7,2,1
2,4,79,4,7,A,2,11,1,50,7,1,0
3,3,29,2,6,B,2,11,2,31,5,2,0
4,3,29,2,4,D,2,11,2,31,5,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1474554,Z,24,X,8,B,Z,11,Z,70,Z,Z,0
1474555,Z,64,X,8,C,Z,11,Z,70,Z,Z,0
1474556,Z,29,X,8,A,Z,11,Z,70,Z,Z,0
1474557,Z,54,X,8,B,Z,11,Z,70,Z,Z,0


In [8]:
def remove_zs(df, column_name):
    df = df[df[column_name] != 'Z']
    return df

In [9]:
typl_df = our_data[['TYPL']]

In [10]:
typl_df

,TYPL
0,2
1,2
2,1
3,2
4,2
...,...
1474554,Z
1474555,Z
1474556,Z
1474557,Z


In [11]:
typl_df = remove_zs(typl_df, 'TYPL')

In [12]:
typl_df

,TYPL
0,2
1,2
2,1
3,2
4,2
...,...
1444289,1
1444290,1
1444291,1
1444292,1


# Scaling

In [13]:
def scaling_ordinal(df: pd.DataFrame, column: str, order_list: t.List=None):
    # get all the unique values for the column
    uniques = df[column].unique()

    # copy the dataframe without the column we are replacing
    df_scaled = df[[c for c in df.columns if c!= column]].copy()

    for unique in uniques:
        if order_list:
            df_scaled[f"{column}<={unique}"] = df[column].apply(order_list.index) <= order_list.index(unique)
        else:
            df_scaled[f"{column}<={unique}"] = df[column] <= unique

    return df_scaled

In [14]:
def scaling_interordinal(df: pd.DataFrame, column: str, order_list=None):
    # get all the unique values for the column
    uniques = df[column].unique()

    # copy the dataframe without the column we are replacing
    df_scaled = df[[c for c in df.columns if c!= column]].copy()

    for unique in uniques:
        if order_list:
            df_scaled[f"{column}<={unique}"] = df[column].apply(order_list.index) <= order_list.index(unique)
            df_scaled[f"{column}>={unique}"] = df[column].apply(order_list.index) >= order_list.index(unique)
        else:
            df_scaled[f"{column}<={unique}"] = df[column] <= unique
            df_scaled[f"{column}>={unique}"] = df[column] >= unique

    return df_scaled

In [15]:
def scaling_nominal(df: pd.DataFrame, column: str):
    # get all the unique values for the column
    uniques = df[column].unique()

    # copy the dataframe without the column we are replacing
    df_scaled = df[[c for c in df.columns if c!= column]].copy()

    for unique in uniques:
        df_scaled[f"{column}={unique}"] = df[column] == unique

    return df_scaled

# All Closure

In [16]:
def up(ctx: pd.DataFrame, objs: t.Iterable) -> t.List:
    objs = ctx.loc[list(objs)] # get the rows of the objects
    attrs = ctx.columns[objs.all(axis=0)]  # get the column name (attribute) when all the values in the column are True
    return attrs.to_list()

def down(ctx: pd.DataFrame, attrs: t.Iterable) -> t.List:
    attrs = ctx[list(attrs)] # get the columns of the attributes
    objs = ctx.index[attrs.all(axis=1)]  # get the row index (object) when all the values in the row are True
    return objs.to_list()

def updown(ctx: pd.DataFrame, objs: t.Iterable) -> t.Set: return set(down(ctx, up(ctx, objs)))
def downup(ctx: pd.DataFrame, attrs: t.Iterable) -> t.Set: return set(up(ctx, down(ctx, attrs)))


In [17]:
def LexicallyLower(intent: t.Set, other: t.Set, attribute_order: t.List, i: int) -> bool:
    if attribute_order[i] not in other.difference(intent):
        return False

    return intent.intersection(attribute_order[:i]) == other.intersection(attribute_order[:i])
    
def NextClosure(ctx: pd.DataFrame, intent: t.Set, attribute_order: t.List) -> t.Set:
    intent = set(intent)
    for i, a in reversed(list(enumerate(attribute_order))): # For all i in M (in reversed order) do
        intent_b = intent.intersection(set(attribute_order[:i]))
        intent_b = downup(ctx, intent_b.union({a}))

        if LexicallyLower(intent, intent_b, attribute_order, i):
            return intent_b
    #return intent # Return A

In [18]:
def AllClosureYield(ctx: pd.DataFrame, attribute_order: t.List) -> t.Generator[t.Set, None, None]:
    intent = downup(ctx, set())
    bottom = set(up(ctx, set()))
    
    while intent != bottom:
        yield intent
        intent = NextClosure(ctx, intent, attribute_order)
    yield intent
def AllClosure(ctx: pd.DataFrame, attribute_order: t.List) -> t.List[t.Set]:
    return [intent for intent in AllClosureYield(ctx, attribute_order)]

# Quesion 1

In [19]:
df = our_data[['AGER20', 'ACHLR']]

In [20]:
df = remove_zs(df, 'ACHLR')

In [21]:
df

,AGER20,ACHLR
0,64,3
1,79,3
2,79,4
3,29,3
4,29,3
...,...,...
1444289,14,5
1444290,54,5
1444291,54,5
1444292,79,4


In [22]:
df['ACHLR'] = np.array([int(x) for x in df['ACHLR']])

In [23]:
df['ACHLR']

0          3
1          3
2          4
3          3
4          3
          ..
1444289    5
1444290    5
1444291    5
1444292    4
1444293    4
Name: ACHLR, Length: 1444294, dtype: int64

In [24]:
df_scaled = scaling_nominal(scaling_interordinal(df, 'AGER20'), 'ACHLR')

In [25]:
len(df_scaled.columns)


33

In [26]:
df_scaled_head = df_scaled.head(100)

In [27]:
df_scaled_head

,AGER20<=64,AGER20>=64,AGER20<=79,AGER20>=79,AGER20<=29,AGER20>=29,AGER20<=2,AGER20>=2,AGER20<=39,AGER20>=39,...,AGER20>=10,AGER20<=19,AGER20>=19,ACHLR=3,ACHLR=4,ACHLR=6,ACHLR=7,ACHLR=5,ACHLR=1,ACHLR=2
0,True,True,True,False,False,True,False,True,False,True,...,True,False,True,True,False,False,False,False,False,False
1,False,True,True,True,False,True,False,True,False,True,...,True,False,True,True,False,False,False,False,False,False
2,False,True,True,True,False,True,False,True,False,True,...,True,False,True,False,True,False,False,False,False,False
3,True,False,True,False,True,True,False,True,True,False,...,True,False,True,True,False,False,False,False,False,False
4,True,False,True,False,True,True,False,True,True,False,...,True,False,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,True,False,True,False,True,False,False,True,True,False,...,True,False,True,False,False,True,False,False,False,False
96,True,False,True,False,False,True,False,True,False,True,...,True,False,True,False,False,True,False,False,False,False
97,True,False,True,False,True,False,False,True,True,False,...,True,True,False,False,False,True,False,False,False,False
98,True,False,True,False,True,False,False,True,True,False,...,True,True,False,False,False,True,False,False,False,False


In [28]:
from latviz_convert import df_to_latviz

In [29]:
df_to_latviz(df_scaled_head, '../latviz/q1_head_latviz.json')

In [30]:
attribute_order = sorted(df_scaled.columns)

In [31]:
attribute_order

['ACHLR=1',
 'ACHLR=2',
 'ACHLR=3',
 'ACHLR=4',
 'ACHLR=5',
 'ACHLR=6',
 'ACHLR=7',
 'AGER20<=10',
 'AGER20<=14',
 'AGER20<=17',
 'AGER20<=19',
 'AGER20<=2',
 'AGER20<=24',
 'AGER20<=29',
 'AGER20<=39',
 'AGER20<=5',
 'AGER20<=54',
 'AGER20<=64',
 'AGER20<=79',
 'AGER20<=80',
 'AGER20>=10',
 'AGER20>=14',
 'AGER20>=17',
 'AGER20>=19',
 'AGER20>=2',
 'AGER20>=24',
 'AGER20>=29',
 'AGER20>=39',
 'AGER20>=5',
 'AGER20>=54',
 'AGER20>=64',
 'AGER20>=79',
 'AGER20>=80']

In [32]:
attribute_order = ['ACHLR<=1',
 'ACHLR<=2',
 'ACHLR<=3',
 'ACHLR<=4',
 'ACHLR<=5',
 'ACHLR<=6',
 'ACHLR<=7',
 'AGER20<=2',
 'AGER20<=5',
 'AGER20<=10',
 'AGER20<=14',
 'AGER20<=17',
 'AGER20<=19',
 'AGER20<=24',
 'AGER20<=29',
 'AGER20<=39',
 'AGER20<=54',
 'AGER20<=64',
 'AGER20<=79',
 'AGER20<=80']

In [33]:
attribute_order

['ACHLR<=1',
 'ACHLR<=2',
 'ACHLR<=3',
 'ACHLR<=4',
 'ACHLR<=5',
 'ACHLR<=6',
 'ACHLR<=7',
 'AGER20<=2',
 'AGER20<=5',
 'AGER20<=10',
 'AGER20<=14',
 'AGER20<=17',
 'AGER20<=19',
 'AGER20<=24',
 'AGER20<=29',
 'AGER20<=39',
 'AGER20<=54',
 'AGER20<=64',
 'AGER20<=79',
 'AGER20<=80']

In [103]:
closures = AllClosure(df_scaled, attribute_order)

KeyboardInterrupt: 

In [ ]:
len(closures)

In [ ]:
closures = {i:list(st) for i, st in enumerate(closures)}

In [125]:
closures

{0: ['AGER20>=2', 'AGER20<=80'],
 1: ['AGER20<=80', 'AGER20>=2', 'AGER20>=5'],
 2: ['AGER20>=10', 'AGER20<=80', 'AGER20>=2', 'AGER20>=5'],
 3: ['AGER20>=10', 'AGER20<=80', 'AGER20>=5', 'AGER20>=14', 'AGER20>=2'],
 4: ['AGER20>=17',
  'AGER20<=80',
  'AGER20>=10',
  'AGER20>=5',
  'AGER20>=14',
  'AGER20>=2'],
 5: ['AGER20>=17',
  'AGER20<=80',
  'AGER20>=10',
  'AGER20>=5',
  'AGER20>=19',
  'AGER20>=14',
  'AGER20>=2'],
 6: ['AGER20>=17',
  'AGER20>=24',
  'AGER20<=80',
  'AGER20>=10',
  'AGER20>=5',
  'AGER20>=19',
  'AGER20>=14',
  'AGER20>=2'],
 7: ['AGER20>=17',
  'AGER20>=24',
  'AGER20<=80',
  'AGER20>=10',
  'AGER20>=29',
  'AGER20>=5',
  'AGER20>=19',
  'AGER20>=14',
  'AGER20>=2'],
 8: ['AGER20>=17',
  'AGER20>=24',
  'AGER20<=80',
  'AGER20>=10',
  'AGER20>=29',
  'AGER20>=5',
  'AGER20>=39',
  'AGER20>=19',
  'AGER20>=14',
  'AGER20>=2'],
 9: ['AGER20>=54',
  'AGER20>=17',
  'AGER20>=24',
  'AGER20<=80',
  'AGER20>=10',
  'AGER20>=29',
  'AGER20>=5',
  'AGER20>=39',
  'AGER

In [126]:
import json

with open('../closures/closures_q1.json', 'w') as f:
    json.dump(closures, f)

In [34]:
def itemset_support(df, itemset):
    num = len(down(df, itemset))
    tot = len(df)
    return num / tot

In [35]:
itemset_support(df_scaled, ['ACHLR=3'])

0.24009031402193737

In [36]:
itemset_support(df_scaled, ['AGER20<=39'])

0.4967188120978139

In [37]:
itemset_support(df_scaled, ['AGER20<=39', 'ACHLR=3'])

0.11238985968230845

In [38]:
def count_confidence(df, it1, it2):
    out_of = itemset_support(df, it1)
    if out_of == 0:
        return 0
    return itemset_support(df, it1 + it2) / out_of

In [39]:
age_house = ['ACHLR=1',
 'ACHLR=2',
 'ACHLR=3',
 'ACHLR=4',
 'ACHLR=5',
 'ACHLR=6',
 'ACHLR=7']

age_person_from = [ 'AGER20>=2',
 'AGER20>=5',
 'AGER20>=10',
 'AGER20>=14',
 'AGER20>=17',
 'AGER20>=19',
 'AGER20>=24',
 'AGER20>=29',
 'AGER20>=39',
 'AGER20>=54',
 'AGER20>=64',
 'AGER20>=79',
 'AGER20>=80'] 

age_person_until = [ 'AGER20<=2',
 'AGER20<=5',
 'AGER20<=10',
 'AGER20<=14',
 'AGER20<=17',
 'AGER20<=19',
 'AGER20<=24',
 'AGER20<=29',
 'AGER20<=39',
 'AGER20<=54',
 'AGER20<=64',
 'AGER20<=79',
 'AGER20<=80'] 

In [40]:
for h in age_house:
    for p1 in age_person_from:
        for p2 in age_person_until:
            conf_level = count_confidence(df_scaled, [h], [p1, p2])
            if conf_level > 0.8:
                print(h, p1, p2, conf_level)

ACHLR=1 AGER20>=2 AGER20<=79 0.9238583428432454
ACHLR=1 AGER20>=2 AGER20<=80 1.0
ACHLR=1 AGER20>=5 AGER20<=79 0.8981285740120604
ACHLR=1 AGER20>=5 AGER20<=80 0.974270231168815
ACHLR=1 AGER20>=10 AGER20<=79 0.8695115334967167
ACHLR=1 AGER20>=10 AGER20<=80 0.9456531906534712
ACHLR=1 AGER20>=14 AGER20<=79 0.8169544584338889
ACHLR=1 AGER20>=14 AGER20<=80 0.8930961155906434
ACHLR=1 AGER20>=17 AGER20<=80 0.848146970235534
ACHLR=1 AGER20>=19 AGER20<=80 0.8117411557817147
ACHLR=2 AGER20>=2 AGER20<=64 0.800262903002629
ACHLR=2 AGER20>=2 AGER20<=79 0.9346409298464093
ACHLR=2 AGER20>=2 AGER20<=80 1.0
ACHLR=2 AGER20>=5 AGER20<=79 0.9048775425487754
ACHLR=2 AGER20>=5 AGER20<=80 0.970236612702366
ACHLR=2 AGER20>=10 AGER20<=79 0.8731216272312162
ACHLR=2 AGER20>=10 AGER20<=80 0.938480697384807
ACHLR=2 AGER20>=14 AGER20<=79 0.8162723121627232
ACHLR=2 AGER20>=14 AGER20<=80 0.8816313823163139
ACHLR=2 AGER20>=17 AGER20<=80 0.8354642313546422
ACHLR=3 AGER20>=2 AGER20<=79 0.9204178093845616
ACHLR=3 AGER20>=

In [41]:
for h in age_house:
    for p1 in age_person_from:
        for p2 in age_person_until:
            conf_level = count_confidence(df_scaled, [p1, p2], [h])
            #if conf_level > 0.5:
            print(p1, p2, h, conf_level)

AGER20>=2 AGER20<=2 ACHLR=1 0.08308330480658867
AGER20>=2 AGER20<=5 ACHLR=1 0.08566962881409249
AGER20>=2 AGER20<=10 ACHLR=1 0.09055756583291513
AGER20>=2 AGER20<=14 ACHLR=1 0.09461034571182134
AGER20>=2 AGER20<=17 ACHLR=1 0.09751283956200137
AGER20>=2 AGER20<=19 ACHLR=1 0.09687061287427218
AGER20>=2 AGER20<=24 ACHLR=1 0.09564845760389296
AGER20>=2 AGER20<=29 ACHLR=1 0.09423690702742882
AGER20>=2 AGER20<=39 ACHLR=1 0.09426992729381328
AGER20>=2 AGER20<=54 ACHLR=1 0.10046741984246957
AGER20>=2 AGER20<=64 ACHLR=1 0.10386181560288457
AGER20>=2 AGER20<=79 ACHLR=1 0.10803347127777879
AGER20>=2 AGER20<=80 ACHLR=1 0.11102933336287488
AGER20>=5 AGER20<=2 ACHLR=1 0
AGER20>=5 AGER20<=5 ACHLR=1 0.08813643958745462
AGER20>=5 AGER20<=10 ACHLR=1 0.09321560539657989
AGER20>=5 AGER20<=14 ACHLR=1 0.09736617867235378
AGER20>=5 AGER20<=17 ACHLR=1 0.10026968903046603
AGER20>=5 AGER20<=19 ACHLR=1 0.09916864408770688
AGER20>=5 AGER20<=24 ACHLR=1 0.09723796593245077
AGER20>=5 AGER20<=29 ACHLR=1 0.09537709088

AGER20>=10 AGER20<=17 ACHLR=2 0.0967180148367667
AGER20>=10 AGER20<=19 ACHLR=2 0.09551813892955914
AGER20>=10 AGER20<=24 ACHLR=2 0.09349614984757977
AGER20>=10 AGER20<=29 ACHLR=2 0.09215850377307594
AGER20>=10 AGER20<=39 ACHLR=2 0.09220861486486487
AGER20>=10 AGER20<=54 ACHLR=2 0.09625776727388603
AGER20>=10 AGER20<=64 ACHLR=2 0.09811142535287881
AGER20>=10 AGER20<=79 ACHLR=2 0.09940240879586199
AGER20>=10 AGER20<=80 ACHLR=2 0.10103637363079357
AGER20>=14 AGER20<=2 ACHLR=2 0
AGER20>=14 AGER20<=5 ACHLR=2 0
AGER20>=14 AGER20<=10 ACHLR=2 0
AGER20>=14 AGER20<=14 ACHLR=2 0.09802135816795686
AGER20>=14 AGER20<=17 ACHLR=2 0.09882362721868893
AGER20>=14 AGER20<=19 ACHLR=2 0.09645426808083232
AGER20>=14 AGER20<=24 ACHLR=2 0.0933819372234896
AGER20>=14 AGER20<=29 ACHLR=2 0.09173662589483293
AGER20>=14 AGER20<=39 ACHLR=2 0.09194039799054352
AGER20>=14 AGER20<=54 ACHLR=2 0.09651868686559302
AGER20>=14 AGER20<=64 ACHLR=2 0.09848644576055926
AGER20>=14 AGER20<=79 ACHLR=2 0.09981556995651512
AGER20>=

AGER20>=17 AGER20<=54 ACHLR=3 0.22497320013152652
AGER20>=17 AGER20<=64 ACHLR=3 0.22610756100699061
AGER20>=17 AGER20<=79 ACHLR=3 0.23221385654382673
AGER20>=17 AGER20<=80 ACHLR=3 0.24118805774985844
AGER20>=19 AGER20<=2 ACHLR=3 0
AGER20>=19 AGER20<=5 ACHLR=3 0
AGER20>=19 AGER20<=10 ACHLR=3 0
AGER20>=19 AGER20<=14 ACHLR=3 0
AGER20>=19 AGER20<=17 ACHLR=3 0
AGER20>=19 AGER20<=19 ACHLR=3 0.21721289915303277
AGER20>=19 AGER20<=24 ACHLR=3 0.21676819155031513
AGER20>=19 AGER20<=29 ACHLR=3 0.21664327552696477
AGER20>=19 AGER20<=39 ACHLR=3 0.22008101653188433
AGER20>=19 AGER20<=54 ACHLR=3 0.2244938427715431
AGER20>=19 AGER20<=64 ACHLR=3 0.22579593672484757
AGER20>=19 AGER20<=79 ACHLR=3 0.23225569468841092
AGER20>=19 AGER20<=80 ACHLR=3 0.24164011054865409
AGER20>=24 AGER20<=2 ACHLR=3 0
AGER20>=24 AGER20<=5 ACHLR=3 0
AGER20>=24 AGER20<=10 ACHLR=3 0
AGER20>=24 AGER20<=14 ACHLR=3 0
AGER20>=24 AGER20<=17 ACHLR=3 0
AGER20>=24 AGER20<=19 ACHLR=3 0
AGER20>=24 AGER20<=24 ACHLR=3 0.2165895208505062
AGER

AGER20>=29 AGER20<=2 ACHLR=4 0
AGER20>=29 AGER20<=5 ACHLR=4 0
AGER20>=29 AGER20<=10 ACHLR=4 0
AGER20>=29 AGER20<=14 ACHLR=4 0
AGER20>=29 AGER20<=17 ACHLR=4 0
AGER20>=29 AGER20<=19 ACHLR=4 0
AGER20>=29 AGER20<=24 ACHLR=4 0
AGER20>=29 AGER20<=29 ACHLR=4 0.2382300884955752
AGER20>=29 AGER20<=39 ACHLR=4 0.22525020532463094
AGER20>=29 AGER20<=54 ACHLR=4 0.22353120794765002
AGER20>=29 AGER20<=64 ACHLR=4 0.2538618505859636
AGER20>=29 AGER20<=79 ACHLR=4 0.2762829475261304
AGER20>=29 AGER20<=80 ACHLR=4 0.2738867932059956
AGER20>=39 AGER20<=2 ACHLR=4 0
AGER20>=39 AGER20<=5 ACHLR=4 0
AGER20>=39 AGER20<=10 ACHLR=4 0
AGER20>=39 AGER20<=14 ACHLR=4 0
AGER20>=39 AGER20<=17 ACHLR=4 0
AGER20>=39 AGER20<=19 ACHLR=4 0
AGER20>=39 AGER20<=24 ACHLR=4 0
AGER20>=39 AGER20<=29 ACHLR=4 0
AGER20>=39 AGER20<=39 ACHLR=4 0.21859971750325644
AGER20>=39 AGER20<=54 ACHLR=4 0.22061604770444684
AGER20>=39 AGER20<=64 ACHLR=4 0.25608804114860456
AGER20>=39 AGER20<=79 ACHLR=4 0.28052702227390514
AGER20>=39 AGER20<=80 ACHLR=

AGER20>=54 AGER20<=80 ACHLR=5 0.13256961889484734
AGER20>=64 AGER20<=2 ACHLR=5 0
AGER20>=64 AGER20<=5 ACHLR=5 0
AGER20>=64 AGER20<=10 ACHLR=5 0
AGER20>=64 AGER20<=14 ACHLR=5 0
AGER20>=64 AGER20<=17 ACHLR=5 0
AGER20>=64 AGER20<=19 ACHLR=5 0
AGER20>=64 AGER20<=24 ACHLR=5 0
AGER20>=64 AGER20<=29 ACHLR=5 0
AGER20>=64 AGER20<=39 ACHLR=5 0
AGER20>=64 AGER20<=54 ACHLR=5 0
AGER20>=64 AGER20<=64 ACHLR=5 0.12451907966750116
AGER20>=64 AGER20<=79 ACHLR=5 0.09899810008064405
AGER20>=64 AGER20<=80 ACHLR=5 0.09041784608933072
AGER20>=79 AGER20<=2 ACHLR=5 0
AGER20>=79 AGER20<=5 ACHLR=5 0
AGER20>=79 AGER20<=10 ACHLR=5 0
AGER20>=79 AGER20<=14 ACHLR=5 0
AGER20>=79 AGER20<=17 ACHLR=5 0
AGER20>=79 AGER20<=19 ACHLR=5 0
AGER20>=79 AGER20<=24 ACHLR=5 0
AGER20>=79 AGER20<=29 ACHLR=5 0
AGER20>=79 AGER20<=39 ACHLR=5 0
AGER20>=79 AGER20<=54 ACHLR=5 0
AGER20>=79 AGER20<=64 ACHLR=5 0
AGER20>=79 AGER20<=79 ACHLR=5 0.07301545268477296
AGER20>=79 AGER20<=80 ACHLR=5 0.06566443248686239
AGER20>=80 AGER20<=2 ACHLR=5 0
A

AGER20>=2 AGER20<=14 ACHLR=7 0.03457118213394877
AGER20>=2 AGER20<=17 ACHLR=7 0.03212313059207339
AGER20>=2 AGER20<=19 ACHLR=7 0.03137729856620101
AGER20>=2 AGER20<=24 ACHLR=7 0.031300171402470184
AGER20>=2 AGER20<=29 ACHLR=7 0.034753361135026216
AGER20>=2 AGER20<=39 ACHLR=7 0.036408849636469065
AGER20>=2 AGER20<=54 ACHLR=7 0.031057562256344973
AGER20>=2 AGER20<=64 ACHLR=7 0.02794728341884709
AGER20>=2 AGER20<=79 ACHLR=7 0.025303265090332344
AGER20>=2 AGER20<=80 ACHLR=7 0.024289375985775748
AGER20>=5 AGER20<=2 ACHLR=7 0
AGER20>=5 AGER20<=5 ACHLR=7 0.041926748228244375
AGER20>=5 AGER20<=10 ACHLR=7 0.035053421557675234
AGER20>=5 AGER20<=14 ACHLR=7 0.030709461248501598
AGER20>=5 AGER20<=17 ACHLR=7 0.028569340089024314
AGER20>=5 AGER20<=19 ACHLR=7 0.02815265802307122
AGER20>=5 AGER20<=24 ACHLR=7 0.028843043204703326
AGER20>=5 AGER20<=29 ACHLR=7 0.03312076212746862
AGER20>=5 AGER20<=39 ACHLR=7 0.035344224683899736
AGER20>=5 AGER20<=54 ACHLR=7 0.030035810721037306
AGER20>=5 AGER20<=64 ACHLR=

# Question 2

In [42]:
df = our_data[['DIPL_15', 'CS1', 'HLML']]

In [43]:
df

,DIPL_15,CS1,HLML
0,A,8,1
1,A,7,1
2,A,7,2
3,B,6,2
4,D,4,2
...,...,...,...
1474554,B,8,Z
1474555,C,8,Z
1474556,A,8,Z
1474557,B,8,Z


In [44]:
df = remove_zs(df, 'HLML')

In [47]:
df = remove_zs(df, 'DIPL_15')

In [49]:
df = remove_zs(df, 'CS1')

In [50]:
df

,DIPL_15,CS1,HLML
0,A,8,1
1,A,7,1
2,A,7,2
3,B,6,2
4,D,4,2
...,...,...,...
1444288,C,8,2
1444290,B,4,2
1444291,D,5,2
1444292,B,7,2


In [51]:
df['CS1'] = np.array([int(x) for x in df['CS1']])

In [52]:
df['HLML'] = np.array([int(x) for x in df['HLML']])

In [65]:
df_scaled = scaling_ordinal(scaling_nominal(scaling_nominal(df, 'HLML'), 'CS1'), 'DIPL_15')

In [66]:
df_scaled

,HLML=1,HLML=2,CS1=8,CS1=7,CS1=6,CS1=4,CS1=3,CS1=5,CS1=2,CS1=1,DIPL_15<=A,DIPL_15<=B,DIPL_15<=D,DIPL_15<=C
0,True,False,True,False,False,False,False,False,False,False,True,True,True,True
1,True,False,False,True,False,False,False,False,False,False,True,True,True,True
2,False,True,False,True,False,False,False,False,False,False,True,True,True,True
3,False,True,False,False,True,False,False,False,False,False,False,True,True,True
4,False,True,False,False,False,True,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444288,False,True,True,False,False,False,False,False,False,False,False,False,True,True
1444290,False,True,False,False,False,True,False,False,False,False,False,True,True,True
1444291,False,True,False,False,False,False,False,True,False,False,False,False,True,False
1444292,False,True,False,True,False,False,False,False,False,False,False,True,True,True


In [67]:
df_to_latviz(df_scaled_head, '../latviz/q2_head_latviz.json')

In [68]:
attribute_order = sorted(df_scaled.columns)

In [69]:
attribute_order

['CS1=1',
 'CS1=2',
 'CS1=3',
 'CS1=4',
 'CS1=5',
 'CS1=6',
 'CS1=7',
 'CS1=8',
 'DIPL_15<=A',
 'DIPL_15<=B',
 'DIPL_15<=C',
 'DIPL_15<=D',
 'HLML=1',
 'HLML=2']

In [70]:
closures = AllClosure(df_scaled, attribute_order)

In [71]:
len(closures)

109

In [72]:
closures = {i:list(st) for i, st in enumerate(closures)}

In [73]:
closures

{0: ['DIPL_15<=D'],
 1: ['DIPL_15<=D', 'HLML=2'],
 2: ['DIPL_15<=D', 'HLML=1'],
 3: ['DIPL_15<=C', 'DIPL_15<=D'],
 4: ['DIPL_15<=C', 'DIPL_15<=D', 'HLML=2'],
 5: ['DIPL_15<=C', 'DIPL_15<=D', 'HLML=1'],
 6: ['DIPL_15<=C', 'DIPL_15<=B', 'DIPL_15<=D'],
 7: ['DIPL_15<=C', 'DIPL_15<=B', 'DIPL_15<=D', 'HLML=2'],
 8: ['DIPL_15<=C', 'DIPL_15<=B', 'DIPL_15<=D', 'HLML=1'],
 9: ['DIPL_15<=B', 'DIPL_15<=C', 'DIPL_15<=A', 'DIPL_15<=D'],
 10: ['DIPL_15<=A', 'DIPL_15<=D', 'DIPL_15<=C', 'HLML=2', 'DIPL_15<=B'],
 11: ['HLML=1', 'DIPL_15<=A', 'DIPL_15<=D', 'DIPL_15<=C', 'DIPL_15<=B'],
 12: ['CS1=8', 'DIPL_15<=D'],
 13: ['CS1=8', 'DIPL_15<=D', 'HLML=2'],
 14: ['CS1=8', 'DIPL_15<=D', 'HLML=1'],
 15: ['CS1=8', 'DIPL_15<=C', 'DIPL_15<=D'],
 16: ['CS1=8', 'DIPL_15<=C', 'DIPL_15<=D', 'HLML=2'],
 17: ['CS1=8', 'DIPL_15<=C', 'DIPL_15<=D', 'HLML=1'],
 18: ['CS1=8', 'DIPL_15<=C', 'DIPL_15<=B', 'DIPL_15<=D'],
 19: ['CS1=8', 'DIPL_15<=D', 'DIPL_15<=C', 'HLML=2', 'DIPL_15<=B'],
 20: ['CS1=8', 'HLML=1', 'DIPL_15<=D',

In [74]:
import json

with open('../closures/closures_q2.json', 'w') as f:
    json.dump(closures, f)